[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/latent-consistency-model-colab/blob/main/wip/lcm_cyberrealistic_controlnet_canny_ip_adapter_full_face_draw_colab.ipynb)

In [ ]:
!pip install git+https://github.com/huggingface/diffusers -U
!pip install -q controlnet-aux transformers accelerate peft gradio==3.50.2
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q git+https://github.com/tencent-ailab/IP-Adapter einops

!mkdir /content/image_encoder
!wget https://huggingface.co/h94/IP-Adapter/raw/main/models/image_encoder/config.json -O /content/image_encoder/config.json
!wget https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors?download=true -O /content/image_encoder/model.safetensors

!mkdir /content/models
!wget https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors?download=true -O /content/models/ip-adapter-full-face_sd15.safetensors

from diffusers import StableDiffusionControlNetPipeline, UNet2DConditionModel, ControlNetModel, LCMScheduler
import torch
from ip_adapter import IPAdapterFull

controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained("ckpt/SDHK", controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None,).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")
ip_model = IPAdapterFull(pipe, "/content/image_encoder", "/content/models/ip-adapter-full-face_sd15.safetensors", "cuda", num_tokens=257)

In [ ]:
import numpy as np
import cv2
from PIL import Image
from diffusers.utils import load_image, make_image_grid
image = load_image("https://raw.githubusercontent.com/tencent-ailab/IP-Adapter/main/assets/images/ai_face2.png")
image.resize((256, 256))

input_image = load_image("https://raw.githubusercontent.com/tencent-ailab/IP-Adapter/main/assets/images/ai_face2.png")
input_image = np.array(input_image)
input_image = cv2.Canny(input_image, 100, 200)
input_image = input_image[:, :, None]
input_image = np.concatenate([input_image, input_image, input_image], axis=2)
canny_image = Image.fromarray(input_image)

images = ip_model.generate(width=512, height=768, pil_image=image, image=canny_image, num_samples=4, num_inference_steps=4, guidance_scale=1.0, prompt="1girl red dress", scale=0.6)
make_image_grid(images, rows=1, cols=4)